In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp, wilcoxon

# Append the analysis folder so that you can import custom packages
import sys, os
sys.path.append(r'{}'.format(os.path.abspath(os.pardir)))

# Custom packages
from wholebrain_tools import aba, dataIO
from wholebrain_tools import graphics as gt


# Instantiate an Atlas object from the aba module 
# The first time you run this it will download the structures.json file from the Allen Institute server
paths = dataIO.pathParser()
structuresFile = paths.structures
A = aba.Atlas(nodes=structuresFile)
DFM = aba.AnatomyDataFrameManager(A)

# 1. Load all data

Loads data for **diffuse fluorescence** and **dots** for a **single channel** and 
creates a multi-index dataframe (df) containing raw measurements for all areas
of the brain at high resolution

In [ ]:
# --------------------------------------------------------------------
searchPath = paths.alldata
channelName = 'wfa'     # 'wfa' or 'pv'
# --------------------------------------------------------------------

df = dataIO.allMiceRegions(searchPath=searchPath, channelName=channelName, normCellIntens=True)
df = DFM.multiIndexDf_from_fineDf(df, verbose=True)

# 2. Mid ontology heatmaps

Aggregate data at mid ontology level

In [ ]:
# Dataframe at mid-ontology
midDf = DFM.regionsDf_to_mid(df, verbose=True, normalize=True)
# Select ONLY control animals (CC)
# midDf = midDf.xs('CTR', axis=1, level='treat')

midDf.head()

## 2.1 Heatmap - WFA Diffuse Fluorescence

In [ ]:
data = midDf.xs('diffuseFluo', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='PuBu', vmin=0, vmax=2, title='WFA Diffuse Fluorescence')

# Save the image if necessary
# plt.savefig("diffuseHeatmap.svg", bbox_inches="tight")

## 2.2 Heatmap - PNN Energy

In [ ]:
data = midDf.xs('energy', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='OrRd', vmin=0, vmax=3, title='PNN Energy')

# Save the image if necessary
# plt.savefig("energyHeatmap.svg", bbox_inches="tight")

## 2.3 Heatmap - PNN Density

In [ ]:
data = midDf.xs('density', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='BuGn', vmin=0, vmax=140, title='PNN Density')

# Save the image if necessary
# plt.savefig("pnnDensity.svg", bbox_inches="tight")

## 2.4 Heatmap - PNN Intensity

In [ ]:
data = midDf.xs('intensity', axis=1, level='params')

# Create the heatmap
axObj = gt.midOntologyHeatmap(data, A, cmap='Purples', vmin=0, vmax=1, title='PNN Intensity')

# Save the image if necessary
# plt.savefig("pnnIntensity.svg", bbox_inches="tight")

# 3. Coarse ontology bar plots
Aggregate data at coarse ontology level

In [ ]:
# Dataframe at coarse ontology
coarseDf = DFM.regionsDf_to_coarse(df, verbose=True, normalize=True)


coarseDf.head()

## 3.1 Barplot - WFA Diffuse Fluorescence

In [ ]:
data = coarseDf.xs('diffuseFluo', axis=1, level='params')

# Plot the data
g = gt.coarseOntologyBarplot(data, A, 
    title='WFA Diffuse Fluorescence',
    cmap='PuBu',
    xlabel='(A.U.)',
    areaNames=True,
    )
ws = []
w_ps =[]
# Display the statistics
areaNames = A.ids_to_names(data.index.tolist())
stat, pval = ttest_1samp(data, 1, axis=1, nan_policy='omit')
for i,area in enumerate(areaNames):
    w,w_p= wilcoxon(data.iloc[i]-1, method = 'auto')
    ws.append(w)
    w_ps.append(w_p)
    print(f'{area:25}\t t: {stat[i]:.3f}\t p:{pval[i]:.4f}\t w: {w:.7f}\t w_p:{w_p:.4f}')

# plt.savefig('coarseDiffFluo.svg',bbox_inches="tight")

## 3.2 Barplot - PNN Energy

In [ ]:
data = coarseDf.xs('energy', axis=1, level='params')

# Plot the data
g = gt.coarseOntologyBarplot(data, A, 
    title='PNN Energy',
    cmap='OrRd',
    xlabel='(A.U.)',
    areaNames=True,
    )

ws = []
w_ps =[]
# Display the statistics
areaNames = A.ids_to_names(data.index.tolist())
stat, pval = ttest_1samp(data, 1, axis=1, nan_policy='omit')
for i,area in enumerate(areaNames):
    w,w_p= wilcoxon(data.iloc[i]-1, method = 'auto')
    
    ws.append(w)
    w_ps.append(w_p)
    print(f'{area:25}\t t: {stat[i]:.3f}\t p:{pval[i]:.4f}\t w: {w:.7f}\t w_p:{w_p:.4f}')

# plt.savefig('coarseDiffFluo.svg',bbox_inches="tight")


# 4. Interactions
Explore the relationship between **WFA Diffuse Fluorecence** and **PNN Energy**

## 4.1 Mid-ontology

In [ ]:
data = midDf.groupby('params', axis=1).mean()

### Plot brain regions for a single major subdivision

In [ ]:
areaName = 'Isocortex'
toPlot = data.loc[A.names_to_ids([areaName])[0]]

ax = gt.metricsCorrelation(toPlot, A, 
    x='diffuseFluo',
    y='energy',
    txtLoc = 'br',
    xlabel = 'WFA Diffuse Fluorescence (A.U.)',
    ylabel = 'PNN Energy (A.U.)',
    title = areaName
    )

### Plot all 12 major subdivisions

In [ ]:
f, axs = plt.subplots(nrows=3, ncols=4, figsize=(16,10), squeeze=True)

for i, ax in enumerate(f.axes):
    thisRegion = A.get_major_divisions_ids()[i]
    toPlot = data.loc[thisRegion]

    gt.metricsCorrelation(toPlot, A,
        ax = ax,
        x='diffuseFluo',
        y='energy',
        txtLoc = 'tl' if i in [1,2,3,4] else 'br',
        xlabel = 'WFA Diffuse Fluorescence (A.U.)' if i==8 else None,
        ylabel = 'PNN Energy (A.U.)' if i==8 else None,
        title = A.ids_to_names([thisRegion])[0],
        fontScaling = 0.9
    )

# plt.savefig('allMidAreasCorrelation.svg',bbox_inches="tight")

## 4.1 Coarse ontology

In [ ]:
# Aggregate data at coarse resolution
data = coarseDf.groupby('params', axis=1).mean()

# Calculate SEM for Diffuse Fluorescence and Energy for displaying errorbars
errors = coarseDf.groupby('params', axis=1).sem()
data['errDiffuse'] = errors['diffuseFluo']
data['errEnergy'] = errors['energy']

### Plot coarse areas with errorbars

In [ ]:
f, ax = plt.subplots(figsize=(6,5))

_ = gt.metricsWithErrors(data.copy(), A,
    ax=ax,
    x='diffuseFluo',
    y='energy',
    err_x='errDiffuse',
    err_y='errEnergy',
    xlabel='WFA Diffuse Fluorescence (A.U.)',
    ylabel='PNN Energy (A.U.)',
    annotations=True,
    )

# plt.savefig('allCoarseAreasCorrelation.svg',bbox_inches="tight")